In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
# this data is based on the output of 1.0.0_Data_Filtering
data = pd.read_csv('../data/filtered_reviews_in_Phonex.csv')
data.head(3)

,Unnamed: 0,review_id,user_id,business_id,stars,text,date,business_name,city,state,categories
0,1,aBWKb49Xfkv1946YN7_SIg,sSPbLBHcEMXaJfoO8zs1bA,poSV39UqEg-gpESXafS9-g,5.0,"Amazing food, drinks, service!\n\nWe started w...",2016-01-17 05:33:14,Angry Crab Shack,Phoenix,AZ,"Restaurants, American (New), Seafood, Cajun/Cr..."
1,2,jCHaWXgppmZjkOdpFltWGA,D5ywfFmwtJxLReqAYlDDmw,poSV39UqEg-gpESXafS9-g,5.0,I couldn't be more excited to have found this ...,2016-01-30 01:13:29,Angry Crab Shack,Phoenix,AZ,"Restaurants, American (New), Seafood, Cajun/Cr..."
2,5,CfueO6B4_WauGRQ0cq9Whg,fhwZh6_7YxWeVEuskKMTcA,poSV39UqEg-gpESXafS9-g,5.0,Loved this place! \nI came for crab legs and w...,2016-05-07 01:19:53,Angry Crab Shack,Phoenix,AZ,"Restaurants, American (New), Seafood, Cajun/Cr..."


In [3]:
def train_valid_test_split(data, m, n):

    '''
    construct rating matrix from data
    the columns of which represent business_id
    the rows of which represent user_id
    the values of whose elements represent the according ratings
    @ data: filterd_reviews 
    @ m: counts of ratings for validation
    @ n: counts of ratings for test
    '''

    # to construct sparse matrix
    # train
    train_user_id = []
    train_business_id = []
    train_stars = []
    # validation
    valid_user_id = []
    valid_business_id = []
    valid_stars = []
    # train + validation
    train_valid_user_id = []
    train_valid_business_id = []
    train_valid_stars = []
    # test
    test_user_id = []
    test_business_id = []
    test_stars = []
    
    user_id_lst = data['user_id'].unique().tolist() # rows of sparse matrix
    busi_id_lst = data['business_id'].unique().tolist() # columns of sparse matrix

    train_sparse_matrix = np.zeros(shape=(len(user_id_lst), len(busi_id_lst)))
    valid_sparse_matrix = np.zeros(shape=(len(user_id_lst), len(busi_id_lst)))
    train_valid_sparse_matrix = np.zeros(shape=(len(user_id_lst), len(busi_id_lst)))
    test_sparse_matrix = np.zeros(shape=(len(user_id_lst), len(busi_id_lst)))

    ranking_df = data[['user_id','business_id','stars','date']].groupby(['user_id'])
    
    for group_name, group_df in ranking_df:
        group_df = group_df.sort_values(by='date')

        # if the len(group_df) > valid_m + test_n, split the group_df as 
        # training set : group_df.iloc[:len(group_df)-m-n, :]
        # validation set : group_df.iloc[len(group_df)-m-n:len(group_df)-n, :]
        # test set : group_df.iloc[len(group_df)-n:, :]

        # otherwise, not split the group_df
        # keep the group_df as training set

        if len(group_df) > m+n: 

            training_set = group_df.iloc[:len(group_df)-m-n, :]
            train_user_id.extend(training_set.loc[:,'user_id'].tolist()) 
            train_business_id.extend(training_set.loc[:,'business_id'].tolist())
            train_stars.extend(training_set.loc[:,'stars'].tolist())

            validation_set = group_df.iloc[len(group_df)-m-n:len(group_df)-n, :]
            valid_user_id.extend(validation_set.loc[:,'user_id'].tolist()) 
            valid_business_id.extend(validation_set.loc[:,'business_id'].tolist())
            valid_stars.extend(validation_set.loc[:,'stars'].tolist())
            
            train_validation_set = group_df.iloc[:len(group_df)-n, :]
            train_valid_user_id.extend(train_validation_set.loc[:,'user_id'].tolist()) 
            train_valid_business_id.extend(train_validation_set.loc[:,'business_id'].tolist())
            train_valid_stars.extend(train_validation_set.loc[:,'stars'].tolist())

            testing_set = group_df.iloc[len(group_df)-n:, :]
            test_user_id.extend(testing_set.loc[:,'user_id'].tolist()) 
            test_business_id.extend(testing_set.loc[:,'business_id'].tolist())
            test_stars.extend(testing_set.loc[:,'stars'].tolist())

        else:
            training_set = group_df
            train_user_id.extend(training_set.loc[:,'user_id'].tolist()) 
            train_business_id.extend(training_set.loc[:,'business_id'].tolist())
            train_stars.extend(training_set.loc[:,'stars'].tolist())

    train_df = pd.DataFrame({'user_id': train_user_id, 'business_id': train_business_id, 'stars': train_stars})
    valid_df = pd.DataFrame({'user_id': valid_user_id, 'business_id': valid_business_id, 'stars': valid_stars})
    train_valid_df = pd.DataFrame({'user_id': train_valid_user_id, 'business_id': train_valid_business_id, 'stars': train_valid_stars})
    test_df = pd.DataFrame({'user_id': test_user_id, 'business_id': test_business_id, 'stars': test_stars})


    for i in range(len(train_df)):
        ratings = train_df.iloc[i, 2] # stars
        row_index = user_id_lst.index(train_df.iloc[i, 0]) # user_id
        column_index = busi_id_lst.index(train_df.iloc[i, 1]) # business_id
        train_sparse_matrix[row_index, column_index] = ratings

    for i in range(len(valid_df)):
        ratings = valid_df.iloc[i, 2] # stars
        row_index = user_id_lst.index(valid_df.iloc[i, 0]) # user_id
        column_index = busi_id_lst.index(valid_df.iloc[i, 1]) # business_id
        valid_sparse_matrix[row_index, column_index] = ratings
        
    for i in range(len(train_valid_df)):
        ratings = train_valid_df.iloc[i, 2] # stars
        row_index = user_id_lst.index(train_valid_df.iloc[i, 0]) # user_id
        column_index = busi_id_lst.index(train_valid_df.iloc[i, 1]) # business_id
        train_valid_sparse_matrix[row_index, column_index] = ratings
        
    for i in range(len(test_df)):
        ratings = test_df.iloc[i, 2] # stars
        row_index = user_id_lst.index(test_df.iloc[i, 0]) # user_id
        column_index = busi_id_lst.index(test_df.iloc[i, 1]) # business_id
        test_sparse_matrix[row_index, column_index] = ratings

    # calculate sparstiy of the matrix
    train_sparsity = 1 - np.count_nonzero(train_sparse_matrix)/ (train_sparse_matrix.shape[0] * train_sparse_matrix.shape[1])
    valid_sparsity = 1 - np.count_nonzero(valid_sparse_matrix)/ (valid_sparse_matrix.shape[0] * valid_sparse_matrix.shape[1])
    train_valid_sparsity = 1 - np.count_nonzero(train_valid_sparse_matrix)/ (train_valid_sparse_matrix.shape[0] * train_valid_sparse_matrix.shape[1])
    test_sparsity = 1 - np.count_nonzero(test_sparse_matrix)/ (test_sparse_matrix.shape[0] * test_sparse_matrix.shape[1])

    train_sparsity *= 100
    valid_sparsity *=100
    train_valid_sparse_matrix *= 100
    test_sparsity *= 100

    print (f'{len(user_id_lst)} users')
    print (f'{len(busi_id_lst)} business')

    print (f'Train_rating_matrix Sparsity: {round(train_sparsity,4)}%')
    print (f'Valid_rating_matrix Sparsity: {round(valid_sparsity,4)}%')
    print(f'Test_rating_matrix Sparsity:  {round(test_sparsity,4)}%')


    return train_sparse_matrix, valid_sparse_matrix, train_valid_sparse_matrix, test_sparse_matrix, \
           train_df, valid_df, train_valid_df, test_df, \
           user_id_lst, busi_id_lst


In [4]:
train_sparse_matrix, valid_sparse_matrix, train_valid_sparse_matrix, test_sparse_matrix, \
           train_df, valid_df, train_valid_df, test_df, \
           user_id_lst, busi_id_lst = train_valid_test_split(data=data, m=1, n=1)


19683 users
1728 business
Train_rating_matrix Sparsity: 99.4691%
Valid_rating_matrix Sparsity: 99.9427%
Test_rating_matrix Sparsity:  99.9427%


In [5]:
train_valid_sparse_matrix = np.vstack((train_sparse_matrix, valid_sparse_matrix))
train_valid_df = pd.concat((train_df, valid_df), axis=0)

np.save('train_sparse_matrix.npy', train_sparse_matrix)
np.save('valid_sparse_matrix.npy', valid_sparse_matrix)
np.save('test_sparse_matrix.npy', test_sparse_matrix)
np.save('train_valid_sparse_matrix.npy', train_valid_sparse_matrix)

train_df.to_pickle('../data/train_df.pkl')
valid_df.to_pickle('../data/valid_df.pkl')
test_df.to_pickle('../data/test_df.pkl')
train_valid_df.to_pickle('../data/train_valid_df.pkl')

In [6]:
test_sparse_matrix.shape

(19683, 1728)

# long format to sparse

In [7]:
def long_format_to_sparse(data, pre_feature):
    '''
    @ pre_feature: the feature represents predict_ratings
    '''
    test_sparse_matrix = np.zeros(shape=(len(user_id_lst), len(busi_id_lst)))
    for i in range(len(data)):
        predict_col_index = data.columns.get_loc(pre_feature)
        predict_ratings = data.iloc[i, predict_col_index]
        row_index = user_id_lst.index(data.iloc[i, 0]) # user_id
        column_index = busi_id_lst.index(data.iloc[i, 1]) # business_id
        
        test_sparse_matrix[row_index, column_index] = predict_ratings
        
    return test_sparse_matrix

In [8]:
# Example

In [9]:
nlp_long_format = pd.read_csv('Predictions_CB_bus.csv')
nlp_sparse = long_format_to_sparse(nlp_long_format, 'prediction_ratings')
nlp_sparse

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
nlp_sparse.shape

(19683, 1728)

# sparse to long format

In [11]:
def sparse_to_long_format(sparse_matrix):

    user_loc_lst = np.nonzero(sparse_matrix)[0]
    busi_loc_lst = np.nonzero(sparse_matrix)[1]
    
    prediction = [nlp_sparse[loc] for loc in zip(user_loc_lst, busi_loc_lst)]
    
    user_id = [user_id_lst[i] for i in user_loc_lst]
    busi_id = [busi_id_lst[i] for i in busi_loc_lst]
    
    long_format = pd.DataFrame({'user_id': user_id,
                               'busi_id': busi_id,
                               'prediction_ratings': prediction})
    
    return long_format

In [12]:
# Example

In [13]:
long_format = sparse_to_long_format(nlp_sparse)
long_format.head(1)

,user_id,busi_id,prediction_ratings
0,sSPbLBHcEMXaJfoO8zs1bA,g3w01guDiTLszguA3vUrSg,3.272727
